In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI  
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
import os

# Load environment variables
google_api_key = "your  key"

# Initialize the SQL database
db = SQLDatabase.from_uri("sqlite:/// "+ "D:\Shantanu_Dev\Inboxone\database.db" )

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, max_tokens=500, google_api_key=google_api_key)

# Create an SQL agent
agent_executor = create_sql_agent(llm, db=db, agent_type="zero-shot-react-description", verbose=True)

# Create a prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Example question
question = "tell me if any call scheduled on August 24"

# Execute the SQL agent to get the query result
query_result = agent_executor({"input": question})

# Initialize the embedding model
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)


# Convert the query result to a list of texts (if needed)
texts = [query_result["output"]]  # Assuming the output is a single string, you can adjust as necessary

# Create a vector store from the query result (if needed)
# Create a vector store from the texts
vectorstore = FAISS.from_texts(texts, embeddings_model)

# Create a retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Get the answer
response = rag_chain.invoke({"input": question})
print(response.get("answer", "No answer available."))


In [3]:
# Sample email data
emails = [
    {
        "subject": "Re: Request for Project KT",
        "snippet": "That would be great, thank you!\r\n\r\nOn Fri, 30 Aug 2024, 11:28 pm Skipper Penguin, <Skipper1905@outlook.com> wrote:\r\nHi Shantanu, I've scheduled a KT session for you on 30-08-2024 at 11:54. Please let me know if this time works for you, or if you'd prefer ",
        "body": "\n\nThat would be great, thank you!\nOn Fri, 30 Aug 2024, 11:28 pm Skipper Penguin, <\nSkipper1905@outlook.com\n> wrote:\nHi Shantanu, I've scheduled a KT session for you on 30-08-2024 at 11:54. Please let me know if this time works for you, or if you'd prefer to reschedule. Best, Shantanu Nimkar \nFrom:\n Shantanu Nimkar <\nshantanunimkar19@gmail.com\n>\nSent:\n Friday, August 30, 2024 5:38:01 PM\nTo:\n \nskipper1905@outlook.com\n <\nskipper1905@outlook.com\n>\nSubject:\n Request for Project KT\n \n \nHi penguin, \nCan you please schedule a KT session for me.  Whenever you are available . As I am recently onboarded to test1 project \nThanks you \nShantanu Nimkar ",
        "date": "2024-08-30 23:29:38"
    },
    {
        "subject": "RE: Request for Project KT",
        "snippet": "Hi Shantanu, I've scheduled a KT session for you on 30-08-2024 at 11:54. Please let me know if this time works for you, or if you'd prefer to reschedule. Best, Shantanu Nimkar\r\n________________________________\r\nFrom: Shantanu Nimkar <shantanunimkar19@gmai",
        "body": "\n\nHi Shantanu, I've scheduled a KT session for you on 30-08-2024 at 11:54. Please let me know if this time works for you, or if you'd prefer to reschedule. Best, Shantanu Nimkar \nFrom:\n Shantanu Nimkar <shantanunimkar19@gmail.com>\nSent:\n Friday, August 30, 2024 5:38:01 PM\nTo:\n skipper1905@outlook.com <skipper1905@outlook.com>\nSubject:\n Request for Project KT\n \n \nHi penguin, \nCan you please schedule a KT session for me.  Whenever you are available . As I am recently onboarded to test1 project \nThanks you \nShantanu Nimkar ",
        "date": "2024-08-30 23:28:56"
    },
    {
        "subject": "Request for Project KT",
        "snippet": "Hi penguin,\r\nCan you please schedule a KT session for me.  Whenever you are available . As I am recently onboarded to test1 project\r\n\r\nThanks you\r\nShantanu Nimkar",
        "body": "\n\nHi penguin,\nCan you please schedule a KT session for me.  Whenever you are available . As I am recently onboarded to test1 project \nThanks you \nShantanu Nimkar ",
        "date": "2024-08-30 23:08:18"
    }
]

In [5]:
# Sort emails by date
from datetime import datetime
emails.sort(key=lambda x: datetime.strptime(x['date'], '%Y-%m-%d %H:%M:%S'))

# Generate a summary
summary = []

for email in emails:
    sender = email['body'].split("\n")[1].strip()
    date = email['date']
    subject = email['subject']
    body_start = email['snippet'][:100]  # A snippet of the body for context

    summary.append(f"On {date}, {sender} sent an email regarding '{subject}'. The message started with: '{body_start}'...")

# Combine summary into a single string
final_summary = "\n".join(summary)

print("Summary of the email conversation:")
print(final_summary)

Summary of the email conversation:
On 2024-08-30 23:08:18,  sent an email regarding 'Request for Project KT'. The message started with: 'Hi penguin,
Can you please schedule a KT session for me.  Whenever you are available . As I am rece'...
On 2024-08-30 23:28:56,  sent an email regarding 'RE: Request for Project KT'. The message started with: 'Hi Shantanu, I've scheduled a KT session for you on 30-08-2024 at 11:54. Please let me know if this '...
On 2024-08-30 23:29:38,  sent an email regarding 'Re: Request for Project KT'. The message started with: 'That would be great, thank you!

On Fri, 30 Aug 2024, 11:28 pm Skipper Penguin, <Skipper1905@outlo'...


In [11]:

from langchain_google_genai import  ChatGoogleGenerativeAI  
# Define the database path as a global variable

GOOGLE_API_KEY = "your api key here"


# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, max_tokens=500, google_api_key=GOOGLE_API_KEY)


system_prompt = (
    "You are an assistant trained to summarize email conversations. "
    "Given a collection of email messages, your task is to generate a concise summary of the conversation, including the key details and the dates of the messages. "
    "Ensure that the summary includes the dates in chronological order and highlights the main points of each email. If necessary, group related emails together and summarize their content in a coherent manner."
    "\n\n"
    "Here is the email conversation:\n"
    f"{final_summary}\n"
    "Based on the above conversation, please provide a summary including the dates."
)
messages = [
    (
        "system",
        system_prompt,
    ),
    ("human", "Please provide a concise summary of the above email conversation."),
]
result = llm.invoke(messages)


print(result.content)

On August 30, 2024, Shantanu requested a KT session from Skipper Penguin. Skipper responded by scheduling the session for the same day at 11:54. Shantanu confirmed the time and expressed gratitude. 

